In [ ]:
import warnings

import geopandas as gpd
import libpysal
import momepy
import osmnx
import os
import configparser
import numpy as np

from bokeh.io import output_notebook
output_notebook()

In [17]:
# Define cities and their respective CRS
cities = {
    "Ariel": "EPSG:2039",
    "Acre, Israel": "EPSG:2039",
    "Harish": "EPSG:2039", 
    "Modiin": "EPSG:2039",   
    "Carmiel": "EPSG:2039", 
    "Holon":  "EPSG:2039",
    "Hadera": "EPSG:2039",
    "Bnei Brak": "EPSG:2039",
    "Herzliya": "EPSG:2039",
    "Rehovot": "EPSG:2039",
    "Haifa": "EPSG:2039",  # Replace with actual local CRS for each city
    "Tel Aviv": "EPSG:2039",
    "Jerusalem": "EPSG:2039",
    "Beer Sheva": "EPSG:2039"
}

In [ ]:
# Read the configuration file
config = configparser.ConfigParser()
config_file = 'config.ini'

# Check if file exists
if not os.path.isfile(config_file):
    # Create a simple config file
    config['Paths'] = {
        'gdb_folder': '/path/to/your/gdb/folder',
        'output_dir': '/path/to/your/output/folder'
    }
    with open(config_file, 'w') as configfile:
        config.write(configfile)
    print(f"Created configuration file '{config_file}'.")

# Read the configuration file
config.read(config_file)
gdb_folder = config['Paths']['gdb_folder']
output_dir = config['Paths']['output_dir']

# Ensure the directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Dictionaries to store data for each city
buildings_data = {}
streets_data = {}
tessellation_data = {}

# Loop through each city calculate or load buildings, streets and tessealtions
for city, local_crs in cities.items():
    print(f"Processing {city}")

    # File paths for the saved data
    buildings_file = os.path.join(output_dir, f"{city}_buildings.geojson")
    tessellation_file = os.path.join(output_dir, f"{city}_tessellation.geojson")
    streets_file = os.path.join(output_dir, f"{city}_streets.geojson")

    # Check if files already exist, if so, load them
    if os.path.exists(buildings_file) and os.path.exists(tessellation_file) and os.path.exists(streets_file):
        print(f"Loading saved data for {city}...")
        buildings = gpd.read_file(buildings_file)
        tessellation = gpd.read_file(tessellation_file)
        streets = gpd.read_file(streets_file)
    else:
        print(f"Fetching and processing data for {city}...")
        # Get buildings and street data
        buildings = osmnx.geometries.geometries_from_place(city, tags={'building': True})
        buildings = buildings[buildings.geom_type == "Polygon"].reset_index(drop=True)
        buildings = buildings[["geometry"]].to_crs(local_crs)
        buildings["uID"] = range(len(buildings))

        osm_graph = osmnx.graph_from_place(city, network_type='drive')
        osm_graph = osmnx.projection.project_graph(osm_graph, to_crs=local_crs)

        streets = osmnx.graph_to_gdfs(
            osm_graph,
            nodes=False,
            edges=True,
            node_geometry=False,
            fill_edge_geometry=True
        )
        streets = momepy.remove_false_nodes(streets)
        streets = streets[["geometry"]]
        streets["nID"] = range(len(streets))

        # Generate tessellation and perform metrics
        limit = momepy.buffered_limit(buildings, 100)
        tessellation = momepy.Tessellation(buildings, "uID", limit, verbose=False, segment=1)
        tessellation = tessellation.tessellation

        buildings = buildings.sjoin_nearest(streets, max_distance=1000, how="left")
        buildings = buildings.drop_duplicates("uID").drop(columns="index_right")
        tessellation = tessellation.merge(buildings[['uID', 'nID']], on='uID', how='left')

        # Save the data for future use
        print(f"Saving data for {city}...")
        buildings.to_file(buildings_file, driver="GeoJSON")
        tessellation.to_file(tessellation_file, driver="GeoJSON")
        streets.to_file(streets_file, driver="GeoJSON")
    
    # Store the data in dictionaries
    buildings_data[city] = buildings
    streets_data[city] = streets
    tessellation_data[city] = tessellation

neighbors distance Q1

In [ ]:
output_df_dir = "/Users/annarubtsov/Desktop/neighbor_distance_Q1"  # Replace with your desired folder path

# Dictionary to store the DataFrame for each city
city_metric_dfs = {}

# Loop through each city to calculate metrics based on stored data
for city in buildings_data.keys():
    print(f"Calculating metrics for {city}...")

    # Retrieve the data for the city from the dictionaries
    buildings = buildings_data[city]
    streets = streets_data[city]
    tessellation = tessellation_data[city]  
    
    queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tessellation, ids="uID", silence_warnings=True)
    tessellation["neighbors"] = momepy.Neighbors(tessellation, queen_1, "uID", weighted=True, verbose=False).series
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        buildings["neighbor_distance"] = momepy.NeighborDistance(buildings, queen_1, "uID", verbose=False).series

    buildings["log_neighbor_distance"] = np.log(buildings["neighbor_distance"] + 1) / np.log(10)

    # Create a DataFrame containing the necessary metrics for this city
    city_metrics_df = buildings[["uID", "neighbor_distance", "log_neighbor_distance"]].copy()

    # Store the DataFrame in the dictionary
    city_metric_dfs[city] = city_metrics_df

    # Display the DataFrame for the current city
    print(f"\nMetrics DataFrame for {city}:")
    print(city_metric_dfs[city].head())

    # Define the output file path using city name and metric
    output_file = os.path.join(output_df_dir, f"{city}_neighbor_distance_Q1.csv")

    # Save the DataFrame as a CSV file
    city_metrics_df.to_csv(output_file, index=False)

    print(f"Saved metrics DataFrame for {city} to {output_file}")


neighbors distance Q3

In [ ]:
output_df_dir = "/Users/annarubtsov/Desktop/neighbor_distance_Q3"  # Replace with your desired folder path

# Dictionary to store the DataFrame for each city
city_metric_dfs = {}

# Loop through each city to calculate metrics based on stored data
for city in buildings_data.keys():
    print(f"Calculating metrics for {city}...")

    # Retrieve the data for the city from the dictionaries
    buildings = buildings_data[city]
    streets = streets_data[city]
    tessellation = tessellation_data[city]  
    
    queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tessellation, ids="uID", silence_warnings=True)
    # Create a higher-order spatial weights matrix for tessellation (considering neighbors up to 3rd order)
    queen_3 = momepy.sw_high(k=3, weights=queen_1)

    # Calculate neighbor distances using the higher-order spatial weights
    buildings['neighbor_distance_q3'] = momepy.NeighborDistance(buildings, queen_3, "uID", verbose=False).series

    # Create a DataFrame containing the necessary metrics for this city
    city_metrics_df = buildings[["uID", "neighbor_distance_q3"]].copy()

    # Store the DataFrame in the dictionary
    city_metric_dfs[city] = city_metrics_df

    # Display the DataFrame for the current city
    print(f"\nMetrics DataFrame for {city}:")
    print(city_metric_dfs[city].head())

    # Define the output file path using city name and metric
    output_file = os.path.join(output_df_dir, f"{city}_neighbor_distance_Q3.csv")

    # Save the DataFrame as a CSV file
    city_metrics_df.to_csv(output_file, index=False)

    print(f"Saved metrics DataFrame for {city} to {output_file}")


shared_walls_length

In [ ]:
output_df_dir = "/Users/annarubtsov/Desktop/shared_walls_length"  # Replace with your desired folder path

# Ensure the output directory exists
if not os.path.exists(output_df_dir):
    os.makedirs(output_df_dir)

# Dictionary to store the DataFrame for each city
city_metric_dfs = {}

# Loop through each city to calculate shared walls length based on stored data
for city in buildings_data.keys():
    print(f"Calculating shared walls length for {city}...")

    # Retrieve the data for the city from the dictionaries
    buildings = buildings_data[city]
    tessellation = tessellation_data[city]

    # Calculate shared walls length for each building
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        buildings["shared_walls_length"] = momepy.SharedWalls(buildings).series

    # Create a DataFrame containing the necessary metrics for this city
    city_metrics_df = buildings[["uID", "shared_walls_length"]].copy()

    # Store the DataFrame in the dictionary
    city_metric_dfs[city] = city_metrics_df

    # Display the DataFrame for the current city
    print(f"\nMetrics DataFrame for {city}:")
    print(city_metric_dfs[city].head())

    # Define the output file path using the city name and metric with '.pkl' extension
    output_file = os.path.join(output_df_dir, f"{city}_shared_walls_length.csv")

    # Save the DataFrame as a pickle file
    city_metrics_df.to_csv(output_file, index=False)


    print(f"Saved metrics DataFrame for {city} as pickle to {output_file}")

Covered Area

In [ ]:
output_df_dir = "/Users/annarubtsov/Desktop/covered_area"  # Replace with your desired folder path

# Dictionary to store the DataFrame for each city
city_metric_dfs = {}

# Loop through each city to calculate metrics based on stored data
for city in buildings_data.keys():
    print(f"Calculating metrics for {city}...")

    # Retrieve the data for the city from the dictionaries
    buildings = buildings_data[city]
    streets = streets_data[city]
    tessellation = tessellation_data[city]  
    
    queen_1 = libpysal.weights.contiguity.Queen.from_dataframe(tessellation, ids="uID", silence_warnings=True)
    # Create a higher-order spatial weights matrix for tessellation (considering neighbors up to 3rd order)
    queen_3 = momepy.sw_high(k=3, weights=queen_1)

    # Calculate covered area
    tessellation["covered_area"] = momepy.CoveredArea(tessellation, queen_3, "uID", verbose=False).series

    # Create a DataFrame containing the necessary metrics for this city
    city_metrics_df = tessellation[["uID", "covered_area"]].copy()

    # Store the DataFrame in the dictionary
    city_metric_dfs[city] = city_metrics_df

    # Display the DataFrame for the current city
    print(f"\nMetrics DataFrame for {city}:")
    print(city_metric_dfs[city].head())

    # Define the output file path using city name and metric
    output_file = os.path.join(output_df_dir, f"{city}_covered_area.csv")

    # Save the DataFrame as a CSV file
    city_metrics_df.to_csv(output_file, index=False)

    print(f"Saved metrics DataFrame for {city} to {output_file}")
